### Import models


In [53]:
# All the imports
import os
import copy
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import sounddevice as sd
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torchvision import transforms


In [54]:
class MonoToColor(nn.Module):
    def __init__(self, num_channels=3):
        super(MonoToColor, self).__init__()
        self.num_channels = num_channels

    def forward(self, tensor):
        return tensor.repeat(self.num_channels, 1, 1)


In [55]:
# # Load the best model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_path = os.path.join(os.getcwd(), "ResNet18_Best.pth") 

model = resnet18(pretrained=False)
model.fc = nn.Linear(512, 10)  # UrbanSound8K의 클래스 개수인 10으로 변경
# model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [56]:
# Set up audio recording parameters
SAMPLE_RATE = 22050
DURATION = 1  # in seconds
CHANNELS = 1  # Mono audio


In [57]:
#GPT4

# Real-time audio prediction
def predict_sound(model, device, transformation, sample_rate):
    # Record audio
    recording = sd.rec(
        int(SAMPLE_RATE * DURATION),
        samplerate=SAMPLE_RATE,
        channels=CHANNELS
    )
    sd.wait()

    # Preprocess the recording
    recording = torch.from_numpy(recording).float()
    recording = transformation(recording)

    # Make the prediction
    recording = recording.to(device)
    outputs = model(recording[None, ...])
    _, predicted = torch.max(outputs, 1)

    return predicted.item()


In [58]:
transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=128),
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80),
    MonoToColor()
])

In [59]:
# Run the real-time audio prediction
print(predict_sound(model, device, transformation, SAMPLE_RATE))

RuntimeError: Argument #4: Padding size should be less than the corresponding input dimension, but got: padding (200, 200) at dimension 2 of input [1, 22050, 1]